In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import folium
from folium import Choropleth

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [2]:
conversion_inversa = {v: k for k, v in conversion_provincias.items()}

#Más adelante hay que droppear esta columna. Es la conversión inversa a nombres de provincia
df['nombre_provincia'] = df['id_provincia'].map(conversion_inversa)

datos_agrupados = df.groupby('id_provincia').agg(
    cantidad_coches=('id_provincia', 'size'),
    media_precio=('precio_contado', 'mean')
).reset_index()

datos_agrupados['nombre_provincia'] = datos_agrupados['id_provincia'].map(conversion_inversa)

cantidad_coches_dict = dict(zip(datos_agrupados['nombre_provincia'], datos_agrupados['cantidad_coches']))
media_precio_dict = dict(zip(datos_agrupados['nombre_provincia'], datos_agrupados['media_precio']))

NameError: name 'conversion_provincias' is not defined

In [ ]:
coordenadas = {
    "Álava": (42.8467, -2.6716),
    "Albacete": (38.9943, -1.8564),
    "Alicante": (38.3452, -0.4810),
    "Almería": (36.8340, -2.4637),
    "Asturias": (43.3619, -5.8494),
    "Ávila": (40.6565, -4.7003),
    "Badajoz": (38.8794, -6.9706),
    "Islas Baleares": (39.5696, 2.6502),
    "Barcelona": (41.3851, 2.1734),
    "Burgos": (42.3439, -3.6969),
    "Cáceres": (39.4763, -6.3722),
    "Cádiz": (36.5271, -6.2886),
    "Cantabria": (43.4623, -3.8099),
    "Castellón": (39.9864, -0.0513),
    "Ciudad Real": (38.9848, -3.9270),
    "Córdoba": (37.8882, -4.7794),
    "La Coruña": (43.3623, -8.4115),
    "Cuenca": (40.0704, -2.1374),
    "Girona": (41.9794, 2.8214),
    "Granada": (37.1773, -3.5986),
    "Guadalajara": (40.6333, -3.1669),
    "Guipúzcoa": (43.3183, -1.9812),
    "Huelva": (37.2614, -6.9447),
    "Huesca": (42.1401, -0.4089),
    "Jaén": (37.7796, -3.7849),
    "León": (42.5987, -5.5671),
    "Lleida": (41.6176, 0.6200),
    "La Rioja": (42.4627, -2.4449),
    "Lugo": (43.0120, -7.5559),
    "Madrid": (40.4168, -3.7038),
    "Málaga": (36.7213, -4.4214),
    "Murcia": (37.9922, -1.1307),
    "Navarra": (42.8125, -1.6458),
    "Orense": (42.3358, -7.8639),
    "Palencia": (42.0095, -4.5270),
    "Las Palmas": (28.1235, -15.4363),
    "Pontevedra": (42.4310, -8.6444),
    "Salamanca": (40.9701, -5.6635),
    "Santa Cruz de Tenerife": (28.4636, -16.2518),
    "Segovia": (40.9429, -4.1088),
    "Sevilla": (37.3891, -5.9845),
    "Soria": (41.7636, -2.4645),
    "Tarragona": (41.1189, 1.2445),
    "Teruel": (40.3440, -1.1069),
    "Toledo": (39.8628, -4.0273),
    "Valencia": (39.4699, -0.3763),
    "Valladolid": (41.6523, -4.7245),
    "Vizcaya": (43.2630, -2.9350),
    "Zamora": (41.5033, -5.7446),
    "Zaragoza": (41.6488, -0.8891),
    "Ceuta": (35.8894, -5.3198),
    "Melilla": (35.2923, -2.9381)}

In [ ]:
datos_agrupados.head()

In [ ]:
lista_provincias = [provincia for provincia in datos_agrupados['nombre_provincia'].unique()]

In [ ]:
m_cantidad_coches = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

for i, row in datos_agrupados.iterrows():
    provincia = row['nombre_provincia']
    cantidad = row['cantidad_coches']
    
    if provincia in coordenadas:
        folium.Circle(
            location=coordenadas[provincia],
            radius=cantidad * 10,
            color='blue',
            fill=True,
            fill_color='YlGnBu',
            fill_opacity=0.7,
            popup=f"{provincia}: {cantidad} coches"
        ).add_to(m_cantidad_coches)

m_cantidad_coches